In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile3 = pd.read_pickle("BBB_Train_Decile3.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile3.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile3, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 3')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 7s 2ms/step - loss: 0.4845 - acc: 0.7791 - val_loss: 0.4795 - val_acc: 0.7630
Epoch 57/75
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4920 - acc: 0.7742 - val_loss: 0.4796 - val_acc: 0.7642
Epoch 58/75
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4909 - acc: 0.7794 - val_loss: 0.4794 - val_acc: 0.7630
Epoch 59/75
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4635 - acc: 0.7933 - val_loss: 0.4664 - val_acc: 0.7837
Epoch 24/75
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4654 - acc: 0.7865 - val_loss: 0.4742 - val_acc: 0.7733
Epoch 25/75
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4704 - acc: 0.7843 - val_loss: 0.4632 - val_acc: 0.7837
Epoch 26/75
3087/3087 [==============================] - 8s 2ms/step - loss: 0.4689 - acc: 0.7865 - val_loss: 0.4803 - val_acc: 0.7824
Epoch 27/75
3087/3087 [==============================] - 8s 3ms/ste

3087/3087 [==============================] - 7s 2ms/step - loss: 0.4981 - acc: 0.7664 - val_loss: 0.4934 - val_acc: 0.7500
Epoch 6/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5002 - acc: 0.7736 - val_loss: 0.4904 - val_acc: 0.7539
Epoch 7/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4859 - acc: 0.7745 - val_loss: 0.4891 - val_acc: 0.7526
Epoch 8/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4904 - acc: 0.7700 - val_loss: 0.4852 - val_acc: 0.7513
Epoch 9/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4863 - acc: 0.7787 - val_loss: 0.4832 - val_acc: 0.7578
Epoch 10/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4897 - acc: 0.7745 - val_loss: 0.4785 - val_acc: 0.7526
Epoch 11/125
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4872 - acc: 0.7736 - val_loss: 0.4834 - val_acc: 0.7500
Epoch 12/125
3087/3087 [==============================] - 7s 2ms/

3087/3087 [==============================] - 7s 2ms/step - loss: 0.5181 - acc: 0.7677 - val_loss: 0.4973 - val_acc: 0.7630
Epoch 14/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5169 - acc: 0.7677 - val_loss: 0.4938 - val_acc: 0.7617
Epoch 15/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5117 - acc: 0.7749 - val_loss: 0.4916 - val_acc: 0.7578
Epoch 16/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5140 - acc: 0.7697 - val_loss: 0.4907 - val_acc: 0.7578
Epoch 17/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5089 - acc: 0.7739 - val_loss: 0.4889 - val_acc: 0.7565
Epoch 18/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5019 - acc: 0.7719 - val_loss: 0.4878 - val_acc: 0.7539
Epoch 19/100
3087/3087 [==============================] - 8s 2ms/step - loss: 0.5049 - acc: 0.7694 - val_loss: 0.4862 - val_acc: 0.7565
Epoch 20/100
3087/3087 [==============================] - 8s 

Epoch 73/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4840 - acc: 0.7836 - val_loss: 0.4701 - val_acc: 0.7785
Epoch 74/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4827 - acc: 0.7781 - val_loss: 0.4701 - val_acc: 0.7811
Epoch 75/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4827 - acc: 0.7807 - val_loss: 0.4702 - val_acc: 0.7785
Epoch 76/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4899 - acc: 0.7781 - val_loss: 0.4701 - val_acc: 0.7798
Epoch 77/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4847 - acc: 0.7794 - val_loss: 0.4702 - val_acc: 0.7785
Epoch 78/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4808 - acc: 0.7872 - val_loss: 0.4701 - val_acc: 0.7785
Epoch 79/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4809 - acc: 0.7813 - val_loss: 0.4701 - val_acc: 0.7811
Epoch 80/100
3087/3087 [========================

3087/3087 [==============================] - 7s 2ms/step - loss: 0.4501 - acc: 0.8001 - val_loss: 0.4586 - val_acc: 0.7940
Epoch 33/50
3087/3087 [==============================] - 8s 2ms/step - loss: 0.4463 - acc: 0.7998 - val_loss: 0.4575 - val_acc: 0.7915
Epoch 34/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4504 - acc: 0.7946 - val_loss: 0.4568 - val_acc: 0.7940
Epoch 35/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4521 - acc: 0.7940 - val_loss: 0.4567 - val_acc: 0.7953
Epoch 36/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4493 - acc: 0.7972 - val_loss: 0.4580 - val_acc: 0.7966

Epoch 00036: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 37/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5198 - acc: 0.7690 - val_loss: 0.5025 - val_acc: 0.7591
Epoch 17/125
3087/3087 [==============================] - 7s 2ms/step - loss: 0.5182 - acc: 0.7700 - val_loss: 0.5019 - 

3087/3087 [==============================] - 7s 2ms/step - loss: 0.4577 - acc: 0.7920 - val_loss: 0.4637 - val_acc: 0.7876
Epoch 35/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4548 - acc: 0.7907 - val_loss: 0.4645 - val_acc: 0.7772

Epoch 00035: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 36/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4549 - acc: 0.7979 - val_loss: 0.4575 - val_acc: 0.7915
Epoch 37/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4515 - acc: 0.7972 - val_loss: 0.4569 - val_acc: 0.7927
Epoch 38/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4523 - acc: 0.7975 - val_loss: 0.4565 - val_acc: 0.7953
Epoch 39/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4496 - acc: 0.7933 - val_loss: 0.4560 - val_acc: 0.7927
Epoch 40/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.4542 - acc: 0.7969 - val_loss: 0.45

In [ ]:
0.8297